In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Dataset
#### In this homework we will use the California Housing Prices data from Kaggle. We'll keep working with the 'median_house_value' variaable, and we'll transform it to a classification task

In [2]:
df_original = pd.read_csv(r'./housing.csv')
df_original.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


# Data Preparation
- #### Fill in the missing values with median.
- #### Create a new column 'rooms_per_household' by dividing 'total_rooms' by 'households'.
- #### Create a new column 'bedrooms_per_room' by dividing 'total_bedrooms' by 'total_rooms'.
- #### Create a new column 'population_per_household' by dividing 'population' by 'households'.

In [3]:
# Since we see the values for 'ocean_proximity' are in all caps and have a space, it's better to make the words 
# lower case and use an un underscore instead of a space
categorical_columns = list(df_original.dtypes[df_original.dtypes == 'object'].index)

for c in categorical_columns:
    df_original[c] = df_original[c].str.lower().str.replace(' ', '_')

df_original.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,near_bay
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,near_bay
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,near_bay
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,near_bay
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,near_bay


In [4]:
df_original.dtypes

longitude             float64
latitude              float64
housing_median_age    float64
total_rooms           float64
total_bedrooms        float64
population            float64
households            float64
median_income         float64
median_house_value    float64
ocean_proximity        object
dtype: object

In [5]:
# Next we will need to see which column has null values.
df_original.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [6]:
# Since 'total_bedrooms' has null values, we will fill in that column with the median.
bedroom_median = df_original.total_bedrooms.median()
df_original = df_original.fillna(bedroom_median)
df_original.isnull().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

In [7]:
df_original['rooms_per_household'] = df_original.total_rooms / df_original.households
df_original['bedrooms_per_room'] = df_original.total_bedrooms / df_original.total_rooms
df_original['population_per_household'] = df_original.population / df_original.households
df_original.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,near_bay,6.984127,0.146591,2.555556
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,near_bay,6.238137,0.155797,2.109842
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,near_bay,8.288136,0.129516,2.802260
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,near_bay,5.817352,0.184458,2.547945
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,near_bay,6.281853,0.172096,2.181467


# Question 1
#### What is the most frequent observation (mode) for the column ocean_proximity?
#### Options:
- #### NEAR BAY
- #### <1H OCEAN
- #### INLAND
- #### NEAR OCEAN

In [8]:
df_original.ocean_proximity.mode()
# By using the mode() function, we see that <1h_ocean is the most frequent observation for ocean_proximity.

0    <1h_ocean
dtype: object

# Question 2
- #### Create the correlation matrix for the numerical features of your train dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.
- #### What are the two features that have the biggest correlation in this dataset?
#### Options:
- #### total_bedrooms and households
- #### total_bedrooms and total_rooms
- #### population and households
- #### population_per_household and total_rooms

#### To asnwer question 2, we first need to create the subset of df_filled that only has numerical values. First we create a copy of df_filled, and drop the categorical variables.

In [9]:
df_numerical = df_original.copy()
del df_numerical['ocean_proximity']
del df_numerical['median_house_value']
df_numerical.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_household,bedrooms_per_room,population_per_household
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,6.984127,0.146591,2.555556
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,6.238137,0.155797,2.109842
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,8.288136,0.129516,2.802260
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,5.817352,0.184458,2.547945
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,6.281853,0.172096,2.181467


#### Next we use the corr() function to get the correlation matrix. Since we want to sort the correlation coefficients, we need to unstack the matrix and then sort. Note that a variable will always have a correlation coefficient of 1 with itself.

In [10]:
df_numerical.corr().unstack().sort_values(ascending=False).head(13).sort_values(ascending=True)

total_bedrooms            households                  0.974366
households                total_bedrooms              0.974366
longitude                 longitude                   1.000000
latitude                  latitude                    1.000000
bedrooms_per_room         bedrooms_per_room           1.000000
rooms_per_household       rooms_per_household         1.000000
median_income             median_income               1.000000
households                households                  1.000000
total_bedrooms            total_bedrooms              1.000000
total_rooms               total_rooms                 1.000000
housing_median_age        housing_median_age          1.000000
population                population                  1.000000
population_per_household  population_per_household    1.000000
dtype: float64

#### Hence we see that total_bedrooms and households have the strongest correlation. Alternatively, since we were given choices above we can also just calculate the correlation between the specific columns directly.

In [11]:
print(df_numerical.corrwith(df_original.households).total_bedrooms)
print()
print(df_numerical.corrwith(df_original.total_rooms).total_bedrooms)
print()
print(df_numerical.corrwith(df_original.households).population)
print()
print(df_numerical.corrwith(df_original.total_rooms).population_per_household)

0.974366293770699

0.9270581965414187

0.9072222660959617

-0.02458065899398803


# Make median_house_value binary
- #### We need to turn the median_house_value variable from numerical to binary.
- #### Let's create a variable above_average which is 1 if the median_house_value is above its mean value and 0 otherwise.

In [12]:
house_value_mean = df_original.median_house_value.mean()
df_original['above_average'] = df_original.median_house_value.map(lambda x: int(x > house_value_mean))
df_original.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household,above_average
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,near_bay,6.984127,0.146591,2.555556,1
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,near_bay,6.238137,0.155797,2.109842,1
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,near_bay,8.288136,0.129516,2.802260,1
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,near_bay,5.817352,0.184458,2.547945,1
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,near_bay,6.281853,0.172096,2.181467,1


# Split the data
- #### Split your data in train/val/test sets with 60/20/20 distribution.
- #### Use skcikit-learn fro that (the train_test_split funciton) and set the seed to 42.
- #### Make sure the target value (median_house_value) is not in your dataframe.

In [13]:
df_log_reg = df_original.copy()
del df_log_reg['median_house_value']

df_full_train, df_test = train_test_split(df_log_reg, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.above_average.values
y_val = df_val.above_average.values
y_test = df_test.above_average.values

# Question 3
- #### Calculate the mutual information score between above_average and ocean_proximity. Use the training set only.
- #### Round it to 2 decimals using round(score, 2)
- #### What is their mutual information score?
#### Options:
- #### 0.26
- #### 0
- #### 0.10
- #### 0.16

#### We can easily find the mutual information score between above_average and ocean_proximity by importing mutual_info_score from sklearn.

In [14]:
score = mutual_info_score(df_train.above_average, df_train.ocean_proximity)
round(score, 2)

0.1

#### We see that the mutual info score between above_average and ocean_proximity is 0.10.

# Question 4
#### Now let's train a logistic regression. Remember that we have one categorical variable ocean_proximity in the data. Include it using one-hot encoding. Fit the model on the training dataset. To make sure the results are reproducible acorss different versions of scikit-learn, fit the model with these parameters:
- #### model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
#### Calculate the accuracy on the validation dataset and round it to 2 decimal digits.
#### Options:
- #### 0.60
- #### 0.72
- #### 0.84
- #### 0.95

In [15]:
del df_train['above_average']
del df_val['above_average']
del df_test['above_average']

In [16]:
df_train.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household
0,-119.67,34.43,39.0,1467.0,381.0,1404.0,374.0,2.3681,<1h_ocean,3.922460,0.259714,3.754011
1,-118.32,33.74,24.0,6097.0,794.0,2248.0,806.0,10.1357,near_ocean,7.564516,0.130228,2.789082
2,-121.62,39.13,41.0,1317.0,309.0,856.0,337.0,1.6719,inland,3.908012,0.234624,2.540059
3,-118.63,34.24,9.0,4759.0,924.0,1884.0,915.0,4.8333,<1h_ocean,5.201093,0.194158,2.059016
4,-122.30,37.52,38.0,2769.0,387.0,994.0,395.0,5.5902,near_ocean,7.010127,0.139762,2.516456


In [17]:
dv = DictVectorizer(sparse=False)
train_dicts = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [18]:
model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [19]:
val_dicts = df_val.to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_pred = model.predict(X_val)
y_pred

array([0, 0, 1, ..., 1, 1, 0], dtype=int64)

In [20]:
accuracy = round(accuracy_score(y_val, y_pred), 2)
accuracy

0.84

#### The accuracy of our logistic regression model is 0.84

# Question 5
- #### Let's find the least useful feature using the 'feature elimination' technique. 
- #### Train a model with all these features using the same paramters as in Q4. Now exclude each feature from this set and train a model without it. Record the accuracy for each model. 
- #### For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
- #### Which of the following has the smallest difference?
 - #### total_rooms
 - #### total_bedrooms
 - #### population
 - #### households

In [21]:
# For this problem we need to iterate through each feature, remove said feature, then train a logistic regression model
# without said feature, and see what the accuracy score is by removing it.

features_list = df_train.columns.values.tolist()

def feature_elimination(features):
    min_diff = {}
    feat = {'total_rooms': 1, 'total_bedrooms': 1, 'population': 1, 'bedrooms': 1}
    for f in features:
        features_test = features.copy()
        features_test.remove(f)
        
        dv = DictVectorizer(sparse=False)
        train_dicts = df_train[features_test].to_dict(orient='records')
        X_train = dv.fit_transform(train_dicts)
        
        model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
        model.fit(X_train, y_train)
        
        val_dicts = df_val[features_list].to_dict(orient='records')
        X_val = dv.transform(val_dicts)
        y_pred = model.predict(X_val)
        
        score = round(accuracy_score(y_val, y_pred), 2)
        
        if f in feat.keys():
            min_diff[f] = round(accuracy - score, 2)
        
        print(f'For {f}, the difference is {round(accuracy - score, 2)} and accuracy score is {score}')
        
    min_key = min(min_diff, key=min_diff.get)
    min_value = min_diff[min_key]
    ans = 'The feature with the smallest difference is {} with a difference of {}'.format(min_key, min_value)
    
    return ans

In [22]:
feature_elimination(features_list)

For longitude, the difference is 0.0 and accuracy score is 0.84
For latitude, the difference is 0.01 and accuracy score is 0.83
For housing_median_age, the difference is 0.01 and accuracy score is 0.83
For total_rooms, the difference is 0.0 and accuracy score is 0.84
For total_bedrooms, the difference is 0.01 and accuracy score is 0.83
For population, the difference is 0.01 and accuracy score is 0.83
For households, the difference is 0.01 and accuracy score is 0.83
For median_income, the difference is 0.05 and accuracy score is 0.79
For ocean_proximity, the difference is 0.02 and accuracy score is 0.82
For rooms_per_household, the difference is 0.0 and accuracy score is 0.84
For bedrooms_per_room, the difference is 0.0 and accuracy score is 0.84
For population_per_household, the difference is 0.0 and accuracy score is 0.84


'The feature with the smallest difference is total_rooms with a difference of 0.0'

# Question 6
#### For this question, we'll see how to use a linear regression model from Scikit-Learn
- #### We'll need to use the original column 'median_house_value'. Apply the logarithmic transformation to this column.
- #### Fit the Ridge regression model (model = Ridge(alpha=a, solver='sag', random_state=42)) on the training data.
- #### This model has a parameter alpha. Let's try the following values: [0, 0.01, 0.1, 1, 10]
- #### Which of these alphas lead to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.

#### If there are multiple options, select the smallest alpha.

In [23]:
# Create function to perform ridge regression and find which alpha gives the best RMSE on the validation set.
def ridge_reg(df):
    df_ridge = df.copy()
    df_ridge.median_house_value = np.log1p(df_ridge.median_house_value)
    a = [0, 0.01, 0.1, 1, 10]
    a_dict = {}
    
    # Split the dataset into training-validation-testing sets
    df_full_train, df_test = train_test_split(df_ridge, test_size=0.2, random_state=42)
    df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

    df_train = df_train.reset_index(drop=True)
    df_val = df_val.reset_index(drop=True)
    df_test = df_test.reset_index(drop=True)

    y_train = df_train.above_average.values
    y_val = df_val.above_average.values
    y_test = df_test.above_average.values

    del df_train['above_average']
    del df_val['above_average']
    del df_test['above_average']    

    dv = DictVectorizer(sparse=False)
    train_dicts = df_train.to_dict(orient='records')
    X_train = dv.fit_transform(train_dicts)
    val_dicts = df_val.to_dict(orient='records')
    X_val = dv.transform(val_dicts)
        
    # Test each number in a to find the best RMSE for our ridge regression model
    for num in a:
        model = Ridge(alpha=num, solver='sag', random_state=42)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        rmse_val = round(mean_squared_error(y_test, y_pred, squared=False), 3)
        a_dict[num] = rmse_val
        print(f'The "a" value of {num} gives an RMSE of {rmse_val}')
        
    min_key = min(a_dict, key=a_dict.get)
    min_value = a_dict[min_key]
    ans = 'The smallest "a" value is {} and gives us a RMSE of {}'.format(min_key, min_value)
    return ans

In [24]:
ridge_reg(df_original)

The "a" value of 0 gives an RMSE of 0.517
The "a" value of 0.01 gives an RMSE of 0.517
The "a" value of 0.1 gives an RMSE of 0.517
The "a" value of 1 gives an RMSE of 0.517
The "a" value of 10 gives an RMSE of 0.517


'The smallest "a" value is 0 and gives us a RMSE of 0.517'